In [95]:
#Import package pandas for data analysis
import pandas as pd
# Import package numpy for numeric computing
import numpy as np
from numpy import int64
from numpy import float64
from numpy import datetime64
# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt
import matplotlib.dates as dates
# Allows plots to appear directly in the notebook.
%matplotlib inline
# For dealing with some Accented characters (in Irish Place names)
import unidecode
# Date/time functionality
import datetime
import time
#for trigonometic calculations
import math
# Check if files exist
from os.path import exists
from os import makedirs
# System specific parameters and functions
import sys
# look at some z-scores for inspecting outliers.
from scipy import stats
import seaborn as sb
# lookup lat/long and convert lat/long to national grid references.
#import geopy
#import pyproj

from patsy import dmatrices
from sklearn import metrics
#binary encoding by using the OneHotEncoder
from sklearn.preprocessing import OneHotEncoder

from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz
#from sklearn.tree import export_text

from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import train_test_split

#import graphviz
#from graphviz import Source
#to read all CSV files in a folder
import os
import glob
import pickle

#### Functions generated at notebooks that might be useful

In [3]:
#method to read chunks from a file and create a dataframe list.
def get_chunks(x):
    """x=[location, dtypes, datecolumns ,date_parser_func, cols]
    exp:mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    exp:dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    exp:cols=['Date time','Temperature','Wind Speed','Precipitation','Cloud Cover']
    exp:x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser, cols]"""
    cwd=os.getcwd()
    path = cwd+x[0]
    chunk_folder = glob.glob(os.path.join(path, "*.csv"))
    chunk_list=[]
    counter=0
    for filename in chunk_folder:
        # read the csv file
        if len(x)==4:
            df_chunk = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        else:
            df_chunk = pd.read_csv(filename,usecols=x[4], low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3])
        chunk_list.append(df_chunk)
        print('Index at chunk_list',counter,' is ', filename, ': ',df_chunk.shape)
        counter+=1
    return chunk_list
#chunk_dbus_list=get_chunks(arg_list)

In [4]:
#method to parse dates
mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")

In [5]:
#method that mathes list index in a dataframe list with id information in dataframe's column. 
#for this method to work id column should be constant. 
def id_list_index_matcher(df_list, column_name):
    df_id_list_index = pd.DataFrame()
    index_count=0
    for df in df_list:
        a=df[column_name].unique()
        df_id_list_index=df_id_list_index.append({'List_index':index_count,column_name:a[0]},ignore_index=True)
        index_count+=1
    return df_id_list_index

In [6]:
#methods that create csv files from dataframe list by naming them with unique value in id column.
#for this method to work id column should be constant. 
def write_to_csv(adress,df_list,id_column ):
    cwd=os.getcwd()
    path = cwd+adress
    for df in df_list:
        a=df[id_column].unique()
        file_name=a[0]
        saved_file_name=file_name+'.csv'
        my_file_bool=os.path.exists(path+saved_file_name)
        if my_file_bool==False:
            df.to_csv(path+saved_file_name,index=False)
            print('Location:', path+saved_file_name)

In [7]:
def write_to_csv_n_r_p(adress,df_list,id_column ):
    """without relative path input"""
    path = adress
    for df in df_list:
        a=df[id_column].unique()
        file_name=a[0]
        saved_file_name=file_name+'.csv'
        my_file_bool=os.path.exists(path+saved_file_name)
        if my_file_bool==False:
            df.to_csv(path+saved_file_name,index=False)
            print('Location:', path+saved_file_name)

In [8]:
#method that takes a dataframe list and prints list lenght and sum of all rows in all dataframes
def df_list_checker(dflist):
    print("number of df in list: ",len(dflist))
    row_count=0
    for df in dflist:
        row_count+=df.shape[0]
    print("number of rows of data in list: ", row_count)

In [9]:
#method to yield(return generator object) dfs for memory efficent concat( to avoid expotential memory use)
#https://stackoverflow.com/questions/70911236/how-can-i-concat-the-huge-pandas-dataframes-into-one
def yield_dfs(x):
    """x=[location, dtypes, datecolumns ,date_parser_func]
    exp:mydateparser = lambda x: pd.to_datetime(x, format="%Y-%m-%d %H:%M:%S")
    exp:dtypes={'TRIPID' :'category','PROGRNUMBER':'int16','STOPPOINTID':'category','PLANNEDTIME_ARR':'int64','PLANNEDTIME_DEP':'int64','ACTUALTIME_ARR':'int64','ACTUALTIME_DEP':'int64','DIFFERENCETIME_ARR':'int64','DIFFERENCETIME_DEP':'int64','LINEID':'category','ROUTEID':'category','DIRECTION':'category','TRIPS_TIME_PROPORTION':'float32','TOTAL_JOURNEY_TIME':'int16','TIME_OF_DAY':'category','DAY_OF_WEEK':'category'}
    exp:x=['/Chunks/DBUS/QP_Implementation',dtypes,['ACTUALTIME_ARR_DATETIME'],mydateparser]"""
    cwd=os.getcwd()
    path = cwd+x[0]
    file_list = glob.glob(os.path.join(path, "*.csv"))
    """generator function to yield dataframes."""
    for filename in file_list:
        df = pd.read_csv(filename, low_memory=False, dtype=x[1],parse_dates=x[2],date_parser=x[3],index_col=False)
        print(df.shape)
        yield df

#df_dbus = pd.concat(yield_dfs(x), axis=0,ignore_index=True)

In [10]:
#method that takes a directory and reads all files inside that directory and its subdirectories 
#creates a dataframe from them and add to dataframe list
def get_chunks_subdirectory(location):
    cwd=os.getcwd()
    adress=location
    path = cwd+adress
    chunk_list=[]
    for root, dirnames, files in os.walk(path):
        print(dirnames)
        print('='*66)
        for mfile in files:
            # read the csv file
            df_chunk = pd.read_csv(os.path.join(root, mfile), low_memory=False)
            chunk_list.append(df_chunk)
            print('Location:', mfile)
    return chunk_list
#chunk_dbus_list_v2=get_chunks_subdirectory('/Chunks/DBUS/LINEID_CHUNKS/')

In [11]:
def write_to_csv_if_in_list(adress,df_list,id_column,IMP_List ):
    cwd=os.getcwd()
    path = cwd+adress
    for df in df_list:
        a=df[id_column].unique()
        if a in IMP_List:
            file_name=a[0]
            saved_file_name=file_name+'.csv'
            my_file_bool=os.path.exists(path+saved_file_name)
            if my_file_bool==False:
                df.to_csv(path+saved_file_name,index=False)
                print('Location:', path+saved_file_name)

In [98]:
def get_sourcefeatures(file, ROUTEID):
    df = pd.read_csv(file, low_memory=False)
    targetrow_df= df[df.ROUTEID == ROUTEID ]
    features_list=[]
    for column in targetrow_df.columns.values:
        if targetrow_df.iloc[0][column]==1:
           features_list.append(column)
    features_list.remove('TOTAL_JOURNEY_TIME')
    return features_list

In [67]:
# Define a Helper function to compute/print all our metrics
def printMetrics(actualVal, predictions):
    #classification evaluation measures
    print('\n==============================================================================')
    print("MAE: ", metrics.mean_absolute_error(actualVal, predictions))
    print("MSE: ", metrics.mean_squared_error(actualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(actualVal, predictions)**0.5)
    print("R²: ", metrics.r2_score(actualVal, predictions))

In [12]:
cols=['ROUTEID','TOTAL_JOURNEY_TIME','PLANNEDTIME_DEP','rain','temp','wet_bulb_temp(C)','vapour_pressure(hPa)','humidity(%)','sea_lvl_pressure(hPa)','sin_hour_of_day','cos_hour_of_day','friday','monday','saturday','sunday','thursday','tuesday','wednesday']
dtypes={'PLANNEDTIME_DEP':'int64','ROUTEID':'category','TOTAL_JOURNEY_TIME':'int16','rain':'float16','temp':'float16','wet_bulb_temp(C)':'float16','vapour_pressure(hPa)':'float16','humidity(%)':'int16','sea_lvl_pressure(hPa)':'float16','sin_hour_of_day':'float16','cos_hour_of_day':'float16','friday':'int16','monday':'int16','saturday':'int16','sunday':'int16','thursday':'int16','tuesday':'int16','wednesday':'int16'}
x=['/Chunks/DBUS/Predictive_Model_Chunks_backup',dtypes,[],mydateparser]

In [13]:
dbus_df_list_v4=get_chunks(x)

Index at chunk_list 0  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/18_3.csv :  (3896, 19)
Index at chunk_list 1  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/67_6.csv :  (4001, 19)
Index at chunk_list 2  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/142_9.csv :  (252, 17)
Index at chunk_list 3  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/17_16.csv :  (307, 16)
Index at chunk_list 4  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/46A_74.csv :  (11186, 19)
Index at chunk_list 5  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/122_14.csv :  (4164, 19)
Index at chunk_list 6  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/150_8.csv :  (4907, 19)
Index at chunk_list 7  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/33X_49.csv :  (181, 17)
Index at chunk_list 8  is  /home/

Index at chunk_list 86  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/69_47.csv :  (592, 18)
Index at chunk_list 87  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/13_66.csv :  (469, 19)
Index at chunk_list 88  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/32_58.csv :  (2233, 19)
Index at chunk_list 89  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/145_91.csv :  (285, 17)
Index at chunk_list 90  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/66_13.csv :  (1448, 19)
Index at chunk_list 91  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/185_56.csv :  (847, 19)
Index at chunk_list 92  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/14_12.csv :  (201, 18)
Index at chunk_list 93  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/83A_23.csv :  (1129, 19)
Index at chunk_list 94  is

Index at chunk_list 158  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/27B_36.csv :  (169, 17)
Index at chunk_list 159  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/1_40.csv :  (4834, 19)
Index at chunk_list 160  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/69_44.csv :  (1267, 19)
Index at chunk_list 161  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/26_26.csv :  (863, 19)
Index at chunk_list 162  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/59_11.csv :  (1451, 19)
Index at chunk_list 163  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/120_7.csv :  (3799, 19)
Index at chunk_list 164  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/151_17.csv :  (4328, 19)
Index at chunk_list 165  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/70_60.csv :  (1869, 19)
Index at chunk_li

Index at chunk_list 247  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/13_63.csv :  (291, 17)
Index at chunk_list 248  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/66B_59.csv :  (846, 18)
Index at chunk_list 249  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/37_15.csv :  (4984, 19)
Index at chunk_list 250  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/145_105.csv :  (8051, 19)
Index at chunk_list 251  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/16_24.csv :  (7436, 19)
Index at chunk_list 252  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/17A_16.csv :  (234, 18)
Index at chunk_list 253  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/33_72.csv :  (870, 19)
Index at chunk_list 254  is  /home/team8/notebook/Pelin/Chunks/DBUS/Predictive_Model_Chunks_backup/38A_33.csv :  (173, 17)
Index at chunk_l

In [15]:
dbus_df_list_v4[0].head(5)

,PLANNEDTIME_DEP,ROUTEID,TOTAL_JOURNEY_TIME,rain,temp,wet_bulb_temp(C),dew_pt_temp(C),vapour_pressure(hPa),humidity(%),sea_lvl_pressure(hPa),sin_hour_of_day,cos_hour_of_day,friday,monday,saturday,sunday,thursday,tuesday,wednesday
0,37010,18_3,2473,0.000000,5.601562,4.300781,2.3,7.199219,79,993.0,0.500000,-0.866211,0,1,0,0,0,0,0
1,41726,18_3,3135,0.000000,7.500000,5.601562,3.0,7.601562,73,995.0,0.258789,-0.965820,0,1,0,0,0,0,0
2,45238,18_3,2446,0.000000,7.500000,5.601562,3.0,7.601562,73,995.0,0.000000,-1.000000,0,1,0,0,0,0,0
3,47426,18_3,3137,0.199951,7.300781,5.898438,3.9,8.101562,78,996.0,-0.258789,-0.965820,0,1,0,0,0,0,0
4,49711,18_3,3307,0.099976,6.398438,5.199219,3.5,7.800781,81,998.5,-0.258789,-0.965820,0,1,0,0,0,0,0


In [99]:
sample_df=dbus_df_list_v4[0]

In [100]:
# Shuffle everything:
sample_df = sample_df.sample(frac=1)

In [101]:
home_path=os.path.normpath(os.getcwd() + os.sep + os.pardir)
file=home_path+'/Lorna/LinearRegression/LR_feature_selection.csv'
get_sourcefeatures(file, '18_3')

['PLANNEDTIME_DEP',
 'rain',
 'humidity(%)',
 'sea_lvl_pressure(hPa)',
 'sin_hour_of_day',
 'cos_hour_of_day',
 'friday',
 'saturday',
 'sunday',
 'thursday',
 'tuesday',
 'wednesday']

In [102]:
source_features = get_sourcefeatures(file, '18_3')

In [103]:
y=sample_df['TOTAL_JOURNEY_TIME']

In [104]:
y.head(2)

3651     867
623     5269
Name: TOTAL_JOURNEY_TIME, dtype: int16

In [105]:
X=sample_df[source_features]

In [106]:
X.head(2)

,PLANNEDTIME_DEP,rain,humidity(%),sea_lvl_pressure(hPa),sin_hour_of_day,cos_hour_of_day,friday,saturday,sunday,thursday,tuesday,wednesday
3651,48107,0.0,72,995.5,-0.258789,-0.965820,0,1,0,0,0,0
623,62639,0.0,67,1017.0,-0.965820,-0.258789,1,0,0,0,0,0


In [107]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

print("Training data:\n", pd.concat([X_train, y_train], axis=1).head(2))
print("\nTest data:\n", pd.concat([X_test, y_test], axis=1).head(2))

Training data:
       PLANNEDTIME_DEP      rain  humidity(%)  sea_lvl_pressure(hPa)  \
730             67066  0.099976           98                  984.0   
1785            66481  0.000000           75                 1017.0   

      sin_hour_of_day  cos_hour_of_day  friday  saturday  sunday  thursday  \
730              -1.0             -0.0       0         0       1         0   
1785             -1.0             -0.0       0         1       0         0   

      tuesday  wednesday  TOTAL_JOURNEY_TIME  
730         0          0                2241  
1785        0          0                3736  

Test data:
       PLANNEDTIME_DEP  rain  humidity(%)  sea_lvl_pressure(hPa)  \
3360            48630   0.0           83                  986.5   
2656            37693   0.5           92                 1019.0   

      sin_hour_of_day  cos_hour_of_day  friday  saturday  sunday  thursday  \
3360        -0.258789        -0.965820       0         1       0         0   
2656         0.500000  

In [108]:
randforr = RandomForestRegressor(n_estimators = 200, random_state = 42, oob_score=True, max_depth=10)
randforr.fit(X_train,y_train)

RandomForestRegressor(max_depth=10, n_estimators=200, oob_score=True,
                      random_state=42)

In [109]:
# Predicted scores for each example. 
randforr_predict_train = randforr.predict(X_train[source_features])
actual_vs_predicted = pd.concat([y_train, pd.DataFrame(randforr_predict_train, columns=['Predicted'], index=y_train.index)], axis=1)
# Show the first ten predictions vs actual values...
actual_vs_predicted.head(10)

,TOTAL_JOURNEY_TIME,Predicted
730,2241,2955.060994
1785,3736,3817.781941
3345,6521,6251.217130
1134,2758,2928.166958
3026,3691,3590.642228
595,4522,4508.326065
750,4376,4791.709030
1280,4740,4555.400943
2075,3226,3040.368412
997,3581,4074.133515


In [110]:
printMetrics(y_train, randforr_predict_train)


MAE:  300.8421602543717
MSE:  183505.69413989937
RMSE:  428.3756460630078
R²:  0.8581171440910921


In [111]:
y_test = df_test['TOTAL_JOURNEY_TIME']
X_test= df_test[source_features]
randforr_predict_test = randforr.predict(X_test)
df_true_vs_predicted_test = pd.DataFrame({'ActualValue': y_test, 'PredictedValue': randforr_predict_test})
display(df_true_vs_predicted_test)
printMetrics(y_test, randforr_predict_test)

,ActualValue,PredictedValue
1812,2999,3567.953682
3676,5058,4992.959355
1767,3921,2915.814447
3483,3973,3767.989797
935,3585,4577.376472
...,...,...
2089,4304,3705.262681
564,4946,5284.327890
400,2696,3172.581706
1248,3974,3687.159899



MAE:  328.97208667548404
MSE:  218614.71505968255
RMSE:  467.56252529440656
R²:  0.8322061689903322


In [113]:
#10 fold cross validations
model=RandomForestRegressor()

In [114]:
accuracyR2 = cross_val_score(model, X, y, scoring='r2', cv = 10)
print("R2s: ", accuracyR2)
print("R2s mean: ",accuracyR2.mean())

R2s:  [0.79527904 0.73399309 0.68243834 0.74739372 0.77103796 0.71644054
 0.77607947 0.75857711 0.76077364 0.71357616]
R2s mean:  0.7455589082525758


In [115]:
accuracyMAE = cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv = 10)
print("MAEs: ", accuracyMAE)
print("MAEs mean: ",accuracyMAE.mean())

MAEs:  [-375.64971795 -402.03648718 -412.25405128 -407.85748718 -409.51484615
 -394.14097436 -391.35953728 -383.10434447 -393.65354756 -386.40125964]
MAEs mean:  -395.59722530485794


In [116]:
accuracyRMSE = cross_val_score(model, X, y, scoring='neg_root_mean_squared_error', cv = 10)
print("RMAEs: ", accuracyRMSE)
print("RMAEs mean: ",accuracyRMSE.mean())

RMAEs:  [-522.22482454 -538.51984905 -748.81352805 -599.46443961 -562.00546681
 -555.57470784 -539.16995658 -551.49412672 -529.3963423  -547.02413408]
RMAEs mean:  -569.3687375573498


In [121]:
measures=['r2','neg_mean_absolute_error','neg_root_mean_squared_error']
accuracy = cross_validate(RandomForestRegressor(n_estimators = 200, random_state = 42, oob_score=True, max_depth=10), X, y, scoring=measures, cv = 3)
accuracy

{'fit_time': array([1.32121086, 1.25317597, 1.23731756]),
 'score_time': array([0.04890966, 0.04691935, 0.04528141]),
 'test_r2': array([0.70797895, 0.73197058, 0.72855288]),
 'test_neg_mean_absolute_error': array([-412.46621183, -415.95931477, -401.34279334]),
 'test_neg_root_mean_squared_error': array([-637.71517525, -584.86082079, -559.38270962])}

In [132]:
routeid=sample_df['ROUTEID'].unique()[0]
modelname='RFR'
MAE = np.mean(accuracy['test_neg_mean_absolute_error'])
RMSE = np.mean(accuracy['test_neg_root_mean_squared_error'])
R2 = np.mean(accuracy['test_r2'])
data=[[modelname,routeid,MAE,RMSE,R2]]
row_df=pd.DataFrame( data, columns=['MODEL','ROUTEID','MAE', 'RMSE','R2'])
row_df

,MODEL,ROUTEID,MAE,RMSE,R2
0,RFR,18_3,-409.922773,-593.986235,0.722834


In [93]:
routeid=sample_df['ROUTEID'].unique()[0]
filename= routeid+'.pkl'
filename

'18_3.pkl'

In [96]:
with open(filename, 'wb') as handle:
    pickle.dump(model, handle, pickle.HIGHEST_PROTOCOL)

In [134]:
home_path=os.path.normpath(os.getcwd() + os.sep + os.pardir)
file=home_path+'/Lorna/LinearRegression/LR_feature_selection.csv'
cwd=os.getcwd()
path = cwd+'/RandomForestRegPickles/'
results_df=pd.DataFrame()
for df in dbus_df_list_v4:
    routeid=df['ROUTEID'].unique()[0]
    df = df.sample(frac=1)
    source_features = get_sourcefeatures(file, routeid)
    y=df['TOTAL_JOURNEY_TIME']
    X=df[source_features]
    randforr = RandomForestRegressor(n_estimators = 200, random_state = 42, oob_score=True, max_depth=10)
    randforr.fit(X,y)
    filename= routeid+'.pkl'
    with open(path+filename, 'wb') as handle:
        pickle.dump(randforr, handle, pickle.HIGHEST_PROTOCOL)
    measures=['r2','neg_mean_absolute_error','neg_root_mean_squared_error']
    accuracy = cross_validate(RandomForestRegressor(n_estimators = 200, random_state = 42, oob_score=True, max_depth=10), X, y, scoring=measures, cv = 3)
    modelname='RFR'
    MAE = np.mean(accuracy['test_neg_mean_absolute_error'])
    RMSE = np.mean(accuracy['test_neg_root_mean_squared_error'])
    R2 = np.mean(accuracy['test_r2'])
    data=[[modelname,routeid,MAE,RMSE,R2]]
    row_df=pd.DataFrame( data, columns=['MODEL','ROUTEID','MAE', 'RMSE','R2'])
    results_df=pd.concat([results_df,row_df],axis=0,ignore_index=True)

In [135]:
results_df.head(5)

,MODEL,ROUTEID,MAE,RMSE,R2
0,RFR,18_3,-410.929076,-592.667635,0.724352
1,RFR,67_6,-402.658558,-571.151409,0.624854
2,RFR,142_9,-220.882134,-379.682732,-0.217300
3,RFR,17_16,-259.992783,-372.625143,0.512902
4,RFR,46A_74,-442.117313,-668.203440,0.566805


In [136]:
results_df.to_csv('RFR_results.csv',index=False)